Using Discrete Fourier Transform for finding the normalized frequency of a random wave and parallelizing the code using CUDA

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
%%cu

#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include <sys/time.h>

#define PI 3.14159265

#define N 10000                                 // the number of random samples
#define thread_count 500                        // the number of threads initialized


__global__ void dft(double *d_samples, double *d_dft_real, double *d_dft_img, double *d_k_by_N, double *d_power_spectrum, double *d_sum_power_spectrum, double *d_sum_freq_power)
{
    int load = N/thread_count;                                        // calculating the load per thread
 
    __shared__ double partial_sum_power_spectrum[thread_count];       // initializing a partial sum variable which will find the partial sums of the power_spectrum for the load assigned to it
    __shared__ double partial_sum_freq_power[thread_count];           // initializing a partial sum variable which will find the partial sums of the freq_power for the load assigned to it

    partial_sum_power_spectrum[threadIdx.x] = 0;                      // initializing the partial sum variables to 0
    partial_sum_freq_power[threadIdx.x] = 0;

    if(threadIdx.x != thread_count-1)                                 // if it is not the last thread, it will do load amt of work
    {
        for(int k=threadIdx.x*load; k<(threadIdx.x+1)*load; k++)      // for every element in the load
        {
            for(int n=0; n<N; n++)
            {
                d_dft_real[k] += d_samples[n] * cos((2*PI*k*n)/N);    // find the real and img part of the dft
                d_dft_img[k] += d_samples[n] * sin((2*PI*k*n)/N);
            }
         
            d_k_by_N[k] = (double)k/N;                                // find the k/N value
         
            d_power_spectrum[k] = d_dft_real[k]*d_dft_real[k] + d_dft_img[k]*d_dft_img[k];          // find the power spectrum value

            partial_sum_power_spectrum[threadIdx.x] += d_power_spectrum[k];                         // obtain the partial power_spectrum sum of the load assigned to the thread
            partial_sum_freq_power[threadIdx.x] += 2*PI*d_k_by_N[k]*d_power_spectrum[k];            // obtain the partial freq_power sum of the load assigned to the thread
        }
    }
    else                                                              // if it is last thread, then it will do load+leftover work
    {
        for(int k=threadIdx.x*load; k<N; k++)
        {
            for(int n=0; n<N; n++)
            {
                d_dft_real[k] += d_samples[n] * cos((2*PI*k*n)/N);
                d_dft_img[k] += d_samples[n] * sin((2*PI*k*n)/N);
            }
         
            d_k_by_N[k] = (double)k/N;
         
            d_power_spectrum[k] = d_dft_real[k]*d_dft_real[k] + d_dft_img[k]*d_dft_img[k];

            partial_sum_power_spectrum[threadIdx.x] += d_power_spectrum[k];
            partial_sum_freq_power[threadIdx.x] += 2*PI*d_k_by_N[k]*d_power_spectrum[k]; 
        }
    }
 
    __syncthreads();
 
    if(threadIdx.x == 0)                                                // if it is the master process
    {
        *d_sum_power_spectrum = 0;                                      // initialize the total sums to 0
        *d_sum_freq_power = 0;

        for(int i=0; i<thread_count; i++)                               // find total sum = sum of partial sums 
        {
            *d_sum_power_spectrum += partial_sum_power_spectrum[i];     
            *d_sum_freq_power += partial_sum_freq_power[i];
        }
    }
}


int main() 
{
    struct timeval t1, t2;                                                                    // for keeping track of the startand end time

    double *samples, *d_samples;                                                              // array which contains the sample values
    double *dft_real, *d_dft_real, *dft_img, *d_dft_img;                                      // array which stores the real and img part of the DFT of the sample
    double *k_by_N, *d_k_by_N, *power_spectrum, *d_power_spectrum;                            // arrays which store the k/N and power spectrum 
    double sum_power_spectrum, *d_sum_power_spectrum, sum_freq_power, *d_sum_freq_power;      // for storing the final sum of the power spectrum and the total frequency
    
    samples = (double*)malloc(sizeof(double) * N);                                            // allocating space for samples in the host
    dft_real = (double*)malloc(sizeof(double) * N);                                           // allocating space for dft_real in the host
    dft_img = (double*)malloc(sizeof(double) * N);                                            // allocating space for dft_img in the host
    k_by_N = (double*)malloc(sizeof(double) * N);                                             // allocating space for k_by_N in the host
    power_spectrum = (double*)malloc(sizeof(double) * N);                                     // allocating space for power_spectrum in the host
 
    cudaMalloc((void**)&d_samples, sizeof(double) * N);                                       // allocating space for d_samples in the device
    cudaMalloc((void**)&d_dft_real, sizeof(double) * N);                                      // allocating space for d_dft_real in the device
    cudaMalloc((void**)&d_dft_img, sizeof(double) * N);                                       // allocating space for d_dft_img in the device
    cudaMalloc((void**)&d_k_by_N, sizeof(double) * N);                                        // allocating space for d_k_by_N in the device
    cudaMalloc((void**)&d_power_spectrum, sizeof(double) * N);                                // allocating space for d_power_spectrum in the device
 
    cudaMalloc((void**)&d_sum_power_spectrum, sizeof(double));                                // allocating space for sum_power_spectrum in the device
    cudaMalloc((void**)&d_sum_freq_power, sizeof(double));                                    // allocating space for sum_freq_power in the device

    for(int i=0; i<N; i++)                                                                    
    {
        samples[i] = ((float)rand()/RAND_MAX)*(float)(2.0) - 1;                               // assigning random values to samples
        // samples[i] = i;
        dft_real[i] = 0;                                                                      // initializing the elements of the other arrays to 0
        dft_img[i] = 0;
        k_by_N[i] = 0;
        power_spectrum[i] = 0;
    }
 
    gettimeofday(&t1, 0);                                                                     // obtaining the start time

    cudaMemcpy(d_samples, samples, sizeof(double) * N, cudaMemcpyHostToDevice);               // copying the host variables to the device variables using cudaMemcpy
    cudaMemcpy(d_dft_real, dft_real, sizeof(double) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_dft_img, dft_img, sizeof(double) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_k_by_N, k_by_N, sizeof(double) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_power_spectrum, power_spectrum, sizeof(double) * N, cudaMemcpyHostToDevice);
 
    dft<<<1,thread_count>>>(d_samples, d_dft_real, d_dft_img, d_k_by_N, d_power_spectrum, d_sum_power_spectrum, d_sum_freq_power);        // function call
 
    cudaMemcpy(&sum_power_spectrum, d_sum_power_spectrum, sizeof(double), cudaMemcpyDeviceToHost);                // copying the results from the device variables to the host variables
    cudaMemcpy(&sum_freq_power, d_sum_freq_power, sizeof(double), cudaMemcpyDeviceToHost);
 
    gettimeofday(&t2, 0);                                                                     // obtaining the end time
 
    float normalized_freq = (float)sum_freq_power/sum_power_spectrum;                         // obtaining the normalized frequency
 
    printf("The normalized frequency of the random wave is %f\n\n", normalized_freq);
 
    double time = (1000000.0*(t2.tv_sec-t1.tv_sec) + t2.tv_usec-t1.tv_usec)/1000000.0;        // calculating the time taken      
    printf("time taken : %lf sec\n", time);
 
    return 0;
}

The normalized frequency of the random wave is 3.141282

time taken : 2.015343 sec

time: 2.9 s
